## Preprocess bulk-RNA seq data (Cho et al., GSE126044)
This notebook processes raw count data from the Cho et al. study


In [1]:
import anndata as ad
import scanpy as sc
import pandas as pd
from rnanorm.datasets import load_toy_data
from rnanorm import FPKM, TPM, CPM, TMM


In [58]:
# Load gene mapping data and compass example data
gene_list_with_synoy = pd.read_table("/Users/z5155527/Desktop/Benchmark-2025-Sep/phase_1_datasets/bulk_RNA-seq/braun_bulk_cleaned/Homo_sapiens.gene_info", sep = '\t')
gene_list_with_synoy = gene_list_with_synoy.loc[:, ["Symbol", "Synonyms"]]
gene_list_with_synoy['Synonyms_separated'] = gene_list_with_synoy['Synonyms'].str.split('|')
gene_list_with_synoy = gene_list_with_synoy.explode('Synonyms_separated')
gene_list_with_synoy = gene_list_with_synoy.drop(columns="Synonyms")

compass_example_data = pd.read_csv("/Users/z5155527/Desktop/Benchmark-2025-Sep/phase_1_datasets/bulk_RNA-seq/compass_gide_tpm.tsv", sep='\t')
compass_example_data_genelist = compass_example_data.columns.tolist()


In [3]:
# code from COMPASS
# load the gene length
gtf_path = '/Users/z5155527/Desktop/Benchmark-2025-Sep/external_validation_datasets/code/gencode.v19.chr_patch_hapl_scaff.annotation.gtf'
import gzip

# Parse the GTF file to create a mapping from gene_id to gene symbol
gene_id_to_symbol = {}

with open(gtf_path, 'r') as gtf_file:
    for line in gtf_file:
        if line.startswith('#'):
            continue
        fields = line.strip().split('\t')
        if fields[2] != 'gene':
            continue
        # Parse attributes field
        attributes_field = fields[8]
        attributes = {}
        for attr in attributes_field.strip().split(';'):
            if attr.strip() == '':
                continue
            key, value = attr.strip().split(' ', 1)
            attributes[key] = value.strip().replace('"', '')
        gene_id = attributes.get('gene_id', None)
        gene_symbol = attributes.get('gene_name', None)
        if gene_id and gene_symbol:
            gene_id_to_symbol[gene_id] = gene_symbol

# Convert to DataFrame for easier use
gene_id_symbol_df = pd.DataFrame(list(gene_id_to_symbol.items()), columns=['gene_id_gencode19', 'gene_symbol'])
gene_id_symbol_df["gene_id"] = gene_id_symbol_df["gene_id_gencode19"].str.split(".", expand=True)[0]

gene_id_symbol_df.head()


,gene_id_gencode19,gene_symbol,gene_id
0,ENSG00000223972.4,DDX11L1,ENSG00000223972
1,ENSG00000227232.4,WASH7P,ENSG00000227232
2,ENSG00000243485.2,MIR1302-11,ENSG00000243485
3,ENSG00000237613.2,FAM138A,ENSG00000237613
4,ENSG00000268020.2,OR4G4P,ENSG00000268020


In [ ]:
# -----------------------------------------------------
# Parse GFF3 (GENCODE v36, GRCh38) and extract mapping from gene_id to gene symbol
# For use in mapping RNA-seq columns (gene ids or gene symbols) to GENCODE annotation
# -----------------------------------------------------

# Path to GFF3 annotation file (GENCODE v36, GRCh38)
gtf_path = '/Users/z5155527/Desktop/Benchmark-2025-Sep/external_validation_datasets/code/Homo_sapiens.GRCh38.115.gff3'

import gzip

# Dictionary to store mapping: gene_id (from 'ID') -> gene_symbol (from 'Name')
gene_id_to_symbol = {}

with open(gtf_path, 'r') as gff3_file:
    for line in gff3_file:
        # Skip comment/header lines
        if line.startswith('#'):
            continue
        fields = line.strip().split('\t')
        # Ignore lines with fewer than 9 fields or non-gene features
        if len(fields) < 9:
            continue
        if fields[2] != 'gene':
            continue
        # Parse the attributes column into a dict
        attributes_field = fields[8]
        attributes = {}
        for attr in attributes_field.strip().split(';'):
            attr = attr.strip()
            # Skip blank or malformed attributes
            if attr == '':
                continue
            if '=' not in attr:
                continue
            key, value = attr.split('=', 1)
            attributes[key] = value
        # Get GFF3-style gene_id and symbol
        gene_id = attributes.get('ID', None)        # e.g. "gene:ENSG00000186092"
        gene_symbol = attributes.get('Name', None)  # e.g. "OR4F5"
        if gene_id and gene_symbol:
            gene_id_to_symbol[gene_id] = gene_symbol

# Convert the mapping dict into a DataFrame for downstream use
gene_id_symbol_df = pd.DataFrame(list(gene_id_to_symbol.items()), columns=['gene_id_gencode36', 'gene_symbol'])
# Remove any version suffix: take only the part before the first dot from 'gene_id_gencode36'
gene_id_symbol_df["gene_id"] = gene_id_symbol_df["gene_id_gencode36"].str.split(".", expand=True)[0]

# Preview resulting id -> symbol table
gene_id_symbol_df.head()


,gene_id_gencode36,gene_symbol,gene_id
0,gene:ENSG00000186092,OR4F5,gene:ENSG00000186092
1,gene:ENSG00000284733,OR4F29,gene:ENSG00000284733
2,gene:ENSG00000284662,OR4F16,gene:ENSG00000284662
3,gene:ENSG00000187634,SAMD11,gene:ENSG00000187634
4,gene:ENSG00000188976,NOC2L,gene:ENSG00000188976


In [80]:
gene_id_symbol_df.to_csv('/Users/z5155527/Desktop/Benchmark-2025-Sep/external_validation_datasets/code/gene_id_symbol_hg38.csv')

In [76]:
# try a 18 sample raw count data on compass
cho_et_al = pd.read_csv("/Users/z5155527/Desktop/Benchmark-2025-Sep/phase_1_datasets/bulk_RNA-seq/raw_count_final/cho_et_al_counts.txt", sep = '\t')
cho_et_al.index = cho_et_al["Unnamed: 0"].tolist()
cho_et_al = cho_et_al.drop(columns="Unnamed: 0").T
cho_et_al


,OR4F5,OR4F29,OR4F16,LINC00115,SAMD11,NOC2L,KLHL17,PLEKHN1,HES4,ISG15,...,PRY,BPY2,DAZ1,DAZ2,CDY1B,BPY2B,DAZ3,DAZ4,BPY2C,CDY1
Dis_01,2,0,0,49,176,6343,339,86,59,52,...,0,0,8,388,0,0,14,8,0,0
Dis_02,0,0,0,26,14,3154,59,18,50,43,...,0,0,0,0,0,0,0,0,0,0
Dis_11,0,0,0,47,5,3656,81,5,16,29,...,0,0,11,31,0,0,161,21,0,0
Dis_12,0,0,0,29,5,3656,362,625,301,1232,...,0,0,0,0,0,0,0,0,0,0
Dis_15,0,0,0,41,25,2665,279,139,79,140,...,0,0,0,0,0,0,0,0,0,0
Dis_04,0,0,0,27,22,983,133,25,19,76,...,0,0,0,0,0,0,0,0,0,0
Dis_16,0,0,0,59,28,4853,228,235,387,248,...,0,0,0,0,0,0,0,0,0,0
Dis_17,0,0,0,43,69,2553,220,96,325,104,...,0,0,0,0,0,0,0,0,0,0
Dis_03,6,0,0,39,62,2609,139,143,192,380,...,0,0,0,0,0,0,0,1,0,0
Dis_07,0,0,0,29,40,2100,80,27,47,33,...,0,0,0,0,0,0,0,0,0,0


In [77]:
cho_et_al.shape

(16, 18747)

In [78]:
# map genes ids to gene symbols
cho_et_al_gene_list = cho_et_al.columns.tolist()
print("Length of cho_et_al_gene_list:", len(cho_et_al_gene_list))
print("Length of gene_id_symbol_df:", len(gene_id_symbol_df))

# Find overlap between cho_et_al_gene_list and gene_id_symbol_df["gene_id"]
gene_id_set = set(gene_id_symbol_df["gene_symbol"])
cho_et_al_gene_set = set(cho_et_al_gene_list)

# Genes in cho_et_al that are also in the GTF gene_id list
overlap_genes = cho_et_al_gene_set & gene_id_set
num_overlap = len(overlap_genes)

# Genes in cho_et_al that could not be mapped (not found in gene_id_symbol_df)
unmapped_genes = cho_et_al_gene_set - gene_id_set
num_unmapped = len(unmapped_genes)

print(f"Number of genes overlapped between cho_et_al and GTF gene_id list: {num_overlap}")
print(f"Number of genes in cho_et_al that could not be mapped: {num_unmapped}")
unmapped_genes

Length of cho_et_al_gene_list: 18747
Length of gene_id_symbol_df: 63568
Number of genes overlapped between cho_et_al and GTF gene_id list: 18720
Number of genes in cho_et_al that could not be mapped: 25


{'1-Dec',
 '1-Mar',
 '1-Sep',
 '10-Mar',
 '10-Sep',
 '11-Mar',
 '11-Sep',
 '12-Sep',
 '14-Sep',
 '2-Mar',
 '2-Sep',
 '3-Mar',
 '3-Sep',
 '4-Mar',
 '4-Sep',
 '5-Mar',
 '5-Sep',
 '6-Mar',
 '6-Sep',
 '7-Mar',
 '7-Sep',
 '8-Mar',
 '8-Sep',
 '9-Mar',
 '9-Sep'}

In [79]:
unmapped_genes_df = pd.DataFrame(unmapped_genes, columns=["unmapped_genes"])
unmapped_genes_df["split_mapped"] = unmapped_genes_df["unmapped_genes"].str.split("-", expand=True)[0]
unmapped_genes_df["split_name"] = unmapped_genes_df["unmapped_genes"].str.split("-", expand=True)[1]
unmapped_genes_df.loc[unmapped_genes_df["split_name"]=="Sep", "new_name"] = "SEPT"
unmapped_genes_df.loc[unmapped_genes_df["split_name"]=="Dec", "new_name"] = "DEC"
unmapped_genes_df.loc[unmapped_genes_df["split_name"]=="Mar", "new_name"] = "MARCH"
unmapped_genes_df["new_gene_name"] = unmapped_genes_df["new_name"] + unmapped_genes_df["split_mapped"]
map_gene_dict = dict(zip(unmapped_genes_df["unmapped_genes"], unmapped_genes_df["new_gene_name"]))
map_gene_dict

{'6-Mar': 'MARCH6',
 '5-Mar': 'MARCH5',
 '11-Sep': 'SEPT11',
 '8-Sep': 'SEPT8',
 '7-Sep': 'SEPT7',
 '1-Dec': 'DEC1',
 '11-Mar': 'MARCH11',
 '1-Sep': 'SEPT1',
 '5-Sep': 'SEPT5',
 '4-Mar': 'MARCH4',
 '8-Mar': 'MARCH8',
 '12-Sep': 'SEPT12',
 '14-Sep': 'SEPT14',
 '9-Sep': 'SEPT9',
 '6-Sep': 'SEPT6',
 '9-Mar': 'MARCH9',
 '1-Mar': 'MARCH1',
 '10-Sep': 'SEPT10',
 '3-Mar': 'MARCH3',
 '10-Mar': 'MARCH10',
 '2-Mar': 'MARCH2',
 '4-Sep': 'SEPT4',
 '3-Sep': 'SEPT3',
 '2-Sep': 'SEPT2',
 '7-Mar': 'MARCH7'}

In [80]:
# rename cho_et_al columns by dictionary
cho_et_al = cho_et_al.rename(columns=map_gene_dict)
cho_et_al


,OR4F5,OR4F29,OR4F16,LINC00115,SAMD11,NOC2L,KLHL17,PLEKHN1,HES4,ISG15,...,PRY,BPY2,DAZ1,DAZ2,CDY1B,BPY2B,DAZ3,DAZ4,BPY2C,CDY1
Dis_01,2,0,0,49,176,6343,339,86,59,52,...,0,0,8,388,0,0,14,8,0,0
Dis_02,0,0,0,26,14,3154,59,18,50,43,...,0,0,0,0,0,0,0,0,0,0
Dis_11,0,0,0,47,5,3656,81,5,16,29,...,0,0,11,31,0,0,161,21,0,0
Dis_12,0,0,0,29,5,3656,362,625,301,1232,...,0,0,0,0,0,0,0,0,0,0
Dis_15,0,0,0,41,25,2665,279,139,79,140,...,0,0,0,0,0,0,0,0,0,0
Dis_04,0,0,0,27,22,983,133,25,19,76,...,0,0,0,0,0,0,0,0,0,0
Dis_16,0,0,0,59,28,4853,228,235,387,248,...,0,0,0,0,0,0,0,0,0,0
Dis_17,0,0,0,43,69,2553,220,96,325,104,...,0,0,0,0,0,0,0,0,0,0
Dis_03,6,0,0,39,62,2609,139,143,192,380,...,0,0,0,0,0,0,0,1,0,0
Dis_07,0,0,0,29,40,2100,80,27,47,33,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# map genes ids to gene symbols again
cho_et_al_gene_list = cho_et_al.columns.tolist()
print("Length of cho_et_al_gene_list:", len(cho_et_al_gene_list))
print("Length of gene_id_symbol_df:", len(gene_id_symbol_df))

# Find overlap between cho_et_al_gene_list and gene_id_symbol_df["gene_id"]
gene_id_set = set(gene_id_symbol_df["gene_symbol"])
cho_et_al_gene_set = set(cho_et_al_gene_list)

# Genes in cho_et_al that are also in the GTF gene_id list
overlap_genes = cho_et_al_gene_set & gene_id_set
num_overlap = len(overlap_genes)

# Genes in cho_et_al that could not be mapped (not found in gene_id_symbol_df)
unmapped_genes = cho_et_al_gene_set - gene_id_set
num_unmapped = len(unmapped_genes)

print(f"Number of genes overlapped between cho_et_al and GTF gene_id list: {num_overlap}")
print(f"Number of genes in cho_et_al that could not be mapped: {num_unmapped}")


Length of cho_et_al_gene_list: 18747
Length of gene_id_symbol_df: 63568
Number of genes overlapped between cho_et_al and GTF gene_id list: 18745
Number of genes in cho_et_al that could not be mapped: 0


In [10]:
# create a dictionary for mapping gene symbols to gene ids
map_gene_dict = dict(zip(gene_id_symbol_df["gene_symbol"], gene_id_symbol_df["gene_id_gencode19"])) # it's actually not 36 but v19 but nvm
# make a copy of cho_et_al and map the new gene names to the old gene names
cho_et_al_copy = cho_et_al.copy()
cho_et_al_copy = cho_et_al_copy.rename(columns=map_gene_dict)
cho_et_al_copy



,ENSG00000186092.4,ENSG00000235249.1,ENSG00000185097.2,ENSG00000225880.4,ENSG00000187634.6,ENSG00000188976.6,ENSG00000187961.9,ENSG00000187583.6,ENSG00000188290.6,ENSG00000187608.5,...,ENSG00000169789.6,ENSG00000183753.5,ENSG00000188120.10,ENSG00000205944.7,ENSG00000172352.5,ENSG00000183795.4,ENSG00000187191.10,ENSG00000205916.6,ENSG00000185894.4,ENSG00000172288.6
Dis_01,2,0,0,49,176,6343,339,86,59,52,...,0,0,8,388,0,0,14,8,0,0
Dis_02,0,0,0,26,14,3154,59,18,50,43,...,0,0,0,0,0,0,0,0,0,0
Dis_11,0,0,0,47,5,3656,81,5,16,29,...,0,0,11,31,0,0,161,21,0,0
Dis_12,0,0,0,29,5,3656,362,625,301,1232,...,0,0,0,0,0,0,0,0,0,0
Dis_15,0,0,0,41,25,2665,279,139,79,140,...,0,0,0,0,0,0,0,0,0,0
Dis_04,0,0,0,27,22,983,133,25,19,76,...,0,0,0,0,0,0,0,0,0,0
Dis_16,0,0,0,59,28,4853,228,235,387,248,...,0,0,0,0,0,0,0,0,0,0
Dis_17,0,0,0,43,69,2553,220,96,325,104,...,0,0,0,0,0,0,0,0,0,0
Dis_03,6,0,0,39,62,2609,139,143,192,380,...,0,0,0,0,0,0,0,1,0,0
Dis_07,0,0,0,29,40,2100,80,27,47,33,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# go through the tpm process using the new gene names 
tpm = TPM(gtf_path).set_output(transform="pandas")
tmm = TMM(m_trim = 0.3, a_trim = 0.05).set_output(transform="pandas")


## counts, TPM, TMM
df_counts = cho_et_al_copy
df_tpm = tpm.fit_transform(df_counts)
df_tmm = tmm.fit_transform(df_counts)
df_tpm


,ENSG00000186092.4,ENSG00000235249.1,ENSG00000185097.2,ENSG00000225880.4,ENSG00000187634.6,ENSG00000188976.6,ENSG00000187961.9,ENSG00000187583.6,ENSG00000188290.6,ENSG00000187608.5,...,ENSG00000169789.6,ENSG00000183753.5,ENSG00000188120.10,ENSG00000205944.7,ENSG00000172352.5,ENSG00000183795.4,ENSG00000187191.10,ENSG00000205916.6,ENSG00000185894.4,ENSG00000172288.6
Dis_01,0.437141,0.0,0.000000,7.465255,11.018411,229.772298,20.035226,6.090966,10.588746,14.674661,...,0.0,0.0,0.348347,18.700776,0.0,0.0,0.796447,0.371055,0.0,0.0
Dis_02,0.000000,0.0,0.000000,4.308439,0.953306,124.268974,3.792666,1.386623,9.760242,13.198703,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
Dis_11,0.000000,0.0,0.000000,7.384866,0.322829,136.585742,4.937144,0.365220,2.961479,8.440321,...,0.0,0.0,0.493982,1.540940,0.0,0.0,9.446066,1.004531,0.0,0.0
Dis_12,0.000000,0.0,0.000000,4.983619,0.353081,149.385156,24.132450,49.930517,60.933667,392.169425,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
Dis_15,0.000000,0.0,0.000000,6.737979,1.688276,104.135167,17.786725,10.619396,15.293851,42.617700,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
Dis_04,0.000000,0.0,0.000000,4.266572,1.428551,36.933737,8.152916,1.836515,3.536819,22.245649,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
Dis_16,0.000000,0.0,0.000000,8.233503,1.605641,161.026498,12.342794,15.245431,63.619109,64.106269,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
Dis_17,0.000000,0.0,0.000000,5.639105,3.718336,79.606216,11.192070,5.852645,50.207557,25.263369,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
Dis_03,1.414224,0.0,0.000000,6.407496,4.185749,101.918312,8.858996,10.921920,37.159421,115.644106,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.050018,0.0,0.0
Dis_07,0.000000,0.0,0.000000,3.944669,2.235786,67.918212,4.221322,1.707322,7.531030,8.314625,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [ ]:
import numpy as np
df_tpm_log = np.log2(df_tpm + 1).copy()
df_tpm_log



In [88]:
# change the gene names back to the old ones 
map_gene_dict_back = dict(zip(gene_id_symbol_df["gene_id_gencode19"], gene_id_symbol_df["gene_id"]))

df_tpm_log_gene_id = df_tpm_log.rename(columns=map_gene_dict_back)
df_tpm_gene_id = df_tpm.rename(columns=map_gene_dict_back)
df_tpm_gene_id.shape




(16, 18747)

In [83]:
gene_id_symbol_df

,gene_id_gencode19,gene_symbol,gene_id
0,ENSG00000223972.4,DDX11L1,ENSG00000223972
1,ENSG00000227232.4,WASH7P,ENSG00000227232
2,ENSG00000243485.2,MIR1302-11,ENSG00000243485
3,ENSG00000237613.2,FAM138A,ENSG00000237613
4,ENSG00000268020.2,OR4G4P,ENSG00000268020
...,...,...,...
63563,ENSG00000241559.2,CU459202.2,ENSG00000241559
63564,ENSG00000264728.1,CU442762.3,ENSG00000264728
63565,ENSG00000238667.1,CU442762.2,ENSG00000238667
63566,ENSG00000238477.1,CU442762.1,ENSG00000238477


In [ ]:
# read compass gene mapping
compass_gene_mapping = pd.read_csv("/Users/z5155527/Desktop/Benchmark-2025-Sep/phase_1_datasets/bulk_RNA-seq/compass/compass_gene_map.csv")
compass_gene_mapping



,ensid,gene_name,ensid_v36,gene_type,gene_supertype,entrezgene
0,ENSG00000121410,A1BG,ENSG00000121410.12,protein_coding,protein_coding,1.0
1,ENSG00000148584,A1CF,ENSG00000148584.15,protein_coding,protein_coding,29974.0
2,ENSG00000175899,A2M,ENSG00000175899.15,protein_coding,protein_coding,2.0
3,ENSG00000166535,A2ML1,ENSG00000166535.20,protein_coding,protein_coding,144568.0
4,ENSG00000128274,A4GALT,ENSG00000128274.17,protein_coding,protein_coding,53947.0
...,...,...,...,...,...,...
15667,ENSG00000203995,ZYG11A,ENSG00000203995.10,protein_coding,protein_coding,440590.0
15668,ENSG00000162378,ZYG11B,ENSG00000162378.13,protein_coding,protein_coding,79699.0
15669,ENSG00000159840,ZYX,ENSG00000159840.16,protein_coding,protein_coding,7791.0
15670,ENSG00000074755,ZZEF1,ENSG00000074755.15,protein_coding,protein_coding,23140.0


In [90]:
# Try to map the columns of df_tpm_gene_id (columns are "gene_id" like "ENSG00000223972")
# to the compass gene mapping (which has an "ensid" column).

# First, create a mapping from compass_gene_mapping 'ensid' to the full row (or to its preferred symbol)
# Let's get both the sets for exploration:
tpm_gene_ids = set(df_tpm_gene_id.columns)
compass_ensids = set(compass_gene_mapping["ensid"])

# Intersect for successfully mappable genes:
common_ensids = tpm_gene_ids & compass_ensids

print(f"Number of genes in TPM data: {len(tpm_gene_ids)}")
print(f"Number of genes in compass mapping: {len(compass_ensids)}")
print(f"Number of genes successfully mapped: {len(common_ensids)}")

# For genes in compass mapping that are not present in TPM data, add columns with 0s
missing_ensids = compass_ensids - tpm_gene_ids

# Add missing columns with zeros (same order as missing_ensids set, sorted for reproducibility)
for ensid in sorted(missing_ensids):
    df_tpm_gene_id[ensid] = 0

# Now reorder the dataframe columns to match the compass_gene_mapping order (if desired)
ordered_ensids = [ensid for ensid in compass_gene_mapping["ensid"] if ensid in df_tpm_gene_id.columns]
df_tpm_gene_id_mapped = df_tpm_gene_id[ordered_ensids].copy()

# Map ensid to gene_name according to compass_gene_mapping
ensid_to_symbol = dict(zip(compass_gene_mapping["ensid"], compass_gene_mapping["gene_name"]))
df_tpm_gene_id_mapped = df_tpm_gene_id_mapped.rename(columns=ensid_to_symbol)

# Optionally: list of unmapped ensids (just for reference)
unmapped_gene_ids = tpm_gene_ids - compass_ensids



Number of genes in TPM data: 18745
Number of genes in compass mapping: 15672
Number of genes successfully mapped: 14766


/var/folders/4k/f164lhy95glg7mpdg8pdz5_m0000gp/T/ipykernel_42843/415242716.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tpm_gene_id[ensid] = 0
/var/folders/4k/f164lhy95glg7mpdg8pdz5_m0000gp/T/ipykernel_42843/415242716.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tpm_gene_id[ensid] = 0
/var/folders/4k/f164lhy95glg7mpdg8pdz5_m0000gp/T/ipykernel_42843/415242716.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance

In [93]:
# Add Index column to df_tpm_gene_id_mapped with the current index as a column
df_tpm_gene_id_mapped["Index"] = df_tpm_gene_id_mapped.index



In [94]:
# clinical data
cho_et_al_clinical = pd.read_csv("/Users/z5155527/Desktop/Benchmark-2025-Sep/external_validation_datasets/bulk_rna_seq/GSE126044/cho_clinical.txt", sep='\t')
cho_et_al_clinical


,ID,Sentrix_ID,Sentrix_Position,Age,Sex,Histology,PD-L1expression,Responsiveness,Immunotherapy_drug,Progression_free_survival(month),Overall_survival(months),Best_response
0,Dis_01,2.019600e+11,R08C01,72,M,Squamous_cell_carcinoma,No,Non-responder,Nivolumab,0.5,1.1,PD
1,Dis_02,2.014380e+11,R03C01,76,M,Squamous_cell_carcinoma,Yes,Responder,Nivolumab,13.8,14.1,PR
2,Dis_03,2.019600e+11,R01C01,65,M,Squamous_cell_carcinoma,NaN,Non-responder,Nivolumab,2.3,11.2,PD
3,Dis_04,2.014140e+11,R07C01,55,F,Adenocarcinoma,Yes,Responder,Nivolumab,13.5,18.7,PR
4,Dis_05,2.014140e+11,R03C01,72,M,Adenocarcinoma,No,Non-responder,Nivolumab,2.8,9.6,PD
5,Dis_06,2.014140e+11,R01C01,54,M,Adenocarcinoma,No,Non-responder,Nivolumab,0.8,0.8,PD
6,Dis_07,2.019600e+11,R04C01,66,M,Squamous_cell_carcinoma,No,Non-responder,Nivolumab,1.1,3.5,PD
7,Dis_08,2.014140e+11,R05C01,34,F,Adenocarcinoma,No,Non-responder,Nivolumab,1.0,1.0,PD
8,Dis_09,2.014140e+11,R02C01,61,M,Squamous_cell_carcinoma,NaN,Non-responder,Nivolumab,0.7,1.0,PD
9,Dis_10,2.019600e+11,R06C01,75,M,Squamous_cell_carcinoma,Yes,Responder,Nivolumab,14.2,14.2,PR


In [95]:
code_16_patients = cho_et_al_clinical.loc[cho_et_al_clinical["Histology"]=="Adenocarcinoma", "ID"]
df_tpm_gene_id_mapped.loc[df_tpm_gene_id_mapped["Index"].isin(code_16_patients), "cancer_code"] = 16
code_17_patients = cho_et_al_clinical.loc[cho_et_al_clinical["Histology"]=="Squamous_cell_carcinoma", "ID"]
df_tpm_gene_id_mapped.loc[df_tpm_gene_id_mapped["Index"].isin(code_17_patients), "cancer_code"] = 17
df_tpm_gene_id_mapped


,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADAT,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,Index,cancer_code
Dis_01,1.853212,0.083577,130.590766,16.155126,22.294196,0.000000,249.039852,9.852181,1.443509,36.952573,...,4.433992,10.285066,34.773127,10.558159,13.586671,19.707506,46.579396,51.253386,Dis_01,17.0
Dis_02,4.467199,0.136357,596.157603,0.627553,37.596731,0.369688,137.230041,19.749891,6.411098,6.881740,...,5.493718,3.174104,49.657755,6.834498,15.336002,90.191252,49.659753,31.374281,Dis_02,17.0
Dis_11,2.892728,0.043098,445.762500,28.137045,8.226709,0.116845,207.433457,35.074108,0.868425,53.034921,...,6.680401,8.478469,70.712316,17.854286,14.113103,62.044355,86.693590,34.963654,Dis_11,16.0
Dis_12,4.067749,0.047136,492.710767,746.689818,151.652395,0.511180,115.022131,14.432811,17.910615,11.843914,...,6.262645,11.079410,35.900684,12.921839,14.911929,88.008075,38.542822,24.421397,Dis_12,17.0
Dis_15,6.321301,0.315539,855.645021,9.839394,32.579546,3.055294,128.303538,9.134237,15.700768,7.841395,...,10.189676,13.820030,30.125258,2.582121,18.636096,191.511343,71.386871,30.034002,Dis_15,16.0
Dis_04,4.519698,0.000000,834.802765,0.911905,11.031522,1.527657,88.013372,3.945226,0.124769,3.141605,...,5.678742,7.161067,18.641710,0.576370,16.576117,311.393866,64.893056,9.818873,Dis_04,16.0
Dis_16,1.192836,0.038277,446.573902,0.125831,59.764061,0.000000,114.767902,12.485603,0.330555,9.247998,...,5.226884,12.843683,13.914668,7.439246,8.707069,226.542333,36.762337,9.018043,Dis_16,16.0
Dis_17,8.062286,0.035971,2649.688076,3.476511,144.309791,0.097523,146.210035,8.931447,5.591460,19.177569,...,15.864199,17.032455,30.800702,1.030250,13.630334,160.958333,73.271285,22.452052,Dis_17,17.0
Dis_03,8.426033,0.540772,165.535935,3.436894,22.130221,1.466129,65.153397,7.464031,6.096932,38.252352,...,12.473648,13.816147,18.999461,1.152408,8.722509,35.358433,16.407416,36.871390,Dis_03,17.0
Dis_07,5.589814,0.093274,2566.753210,0.637779,28.791825,0.101153,52.340049,6.123954,2.255390,16.405859,...,5.266868,7.594010,12.028861,0.419805,12.152271,85.146646,42.749114,19.926804,Dis_07,17.0


In [97]:
df_tpm_gene_id_mapped = df_tpm_gene_id_mapped[compass_example_data_genelist]


df_tpm_gene_id_mapped.to_csv("/Users/z5155527/Desktop/Benchmark-2025-Sep/external_validation_datasets/bulk_rna_seq/GSE126044/cho_et_al_cleaned_TPM.tsv", sep='\t', index=False)


In [ ]:
# change the gene names back to the old ones 
map_gene_dict_back = dict(zip(gene_id_symbol_df["gene_id_gencode19"], gene_id_symbol_df["gene_symbol"]))

cho_et_al_tpm_log = df_tpm_log.rename(columns=map_gene_dict_back)

cho_et_al_tpm = df_tpm.rename(columns=map_gene_dict_back)
cho_et_al_tpm




In [60]:
len(map_gene_dict_back)

63568

In [16]:
cho_et_al_tpm_log.to_csv('/Users/z5155527/Desktop/Benchmark-2025-Sep/phase_1_datasets/bulk_RNA-seq/cho_et_al_tpm_log.tsv', sep='\t')
cho_et_al_tpm.to_csv('/Users/z5155527/Desktop/Benchmark-2025-Sep/phase_1_datasets/bulk_RNA-seq/cho_et_al_tpm.tsv', sep='\t')


In [25]:
pd.DataFrame(unmapped_genes).to_csv("/Users/z5155527/Desktop/Benchmark-2025-Sep/external_validation_datasets/bulk_rna_seq/GSE126044/unmapped_genes.txt", sep='\t', index=False)

In [70]:
# firstly, compare the two gene lists and look for which genes are in compass example data but not in our data
cho_et_al_columns = cho_et_al_tpm.columns.tolist()
gene_list = list(set(compass_example_data_genelist) - set(cho_et_al_columns))
gene_list


['SAMD1',
 'CENPU',
 'C19orf84',
 'SF3B6',
 'AK6',
 'RFX7',
 'RYBP',
 'FTCDNL1',
 'GGT2',
 'ANKRD36C',
 'PLSCR3',
 'C3orf49',
 'NEURL1',
 'LKAAEAR1',
 'JADE2',
 'CEBPZOS',
 'C8orf88',
 'AZI2',
 'RAB29',
 'NAA38',
 'MS4A4E',
 'PIDD1',
 'ARPIN',
 'SMIM24',
 'CCDC184',
 'TMIGD3',
 'MTERF3',
 'GCNT7',
 'RSRP1',
 'ERVMER34-1',
 'ZNF670-ZNF695',
 'GCKR',
 'MTURN',
 'MUC2',
 'CEP131',
 'NOL4L',
 'NPIPB5',
 'LRRC75B',
 'LHFPL4',
 'CIPC',
 'ERICH3',
 'MAST1',
 'PRMT9',
 'RPEL1',
 'LRRC75A',
 'PRR29',
 'OTULIN',
 'HGH1',
 'SARAF',
 'ICE2',
 'NIFK',
 'ERICH5',
 'TMEM262',
 'MYO15B',
 'ZNF738',
 'NPIPB11',
 'RITA1',
 'MTERF1',
 'MALRD1',
 'COL6A5',
 'ANKRD18B',
 'CXCL8',
 'PROSER3',
 'CAD',
 'ACKR1',
 'SLC37A4',
 'ZPR1',
 'LMNTD1',
 'NPIPB4',
 'CT55',
 'ZBED9',
 'TMEM263',
 'HMCN2',
 'KDF1',
 'CEP162',
 'SPATA45',
 'NAPRT',
 'MTCL1',
 'Index',
 'DRICH1',
 'cancer_code',
 'CCDC185',
 'ZNF598',
 'NT5DC4',
 'ATG101',
 'C1orf167',
 'MCEMP1',
 'ZNF852',
 'CEMIP',
 'ZCCHC8',
 'RTP5',
 'ZGRF1',
 'MTERF2'

In [67]:
gene_list_with_synoy[gene_list_with_synoy["Symbol"]=="PIDD1"]

,Symbol,Synonyms_separated
12765,PIDD1,LRDD
12765,PIDD1,MRT75
12765,PIDD1,PIDD


In [72]:
cho_et_al_tpm["PIDD"]

KeyError: 'PIDD'

In [43]:
# then map this gene list to synonym
gene_list_df1_1 = gene_list_with_synoy[gene_list_with_synoy["Symbol"].isin(gene_list)]
gene_list_df1_1


,Symbol,Synonyms_separated
645,CAD,CDG1Z
645,CAD,DEE50
645,CAD,EIEE50
645,CAD,GATD4
2040,ACKR1,CCBP1
...,...,...
40885,AK6,AD-004
40885,AK6,CGI-137
40885,AK6,CINAP
40885,AK6,CIP


In [39]:
gene_list_2_1 = list(set(gene_list) - set(gene_list_df1_1["Symbol"]))
print("length of gene_list_2_1", len(gene_list_2_1))
# then map this gene list to synonym
gene_list_df2_1 = gene_list_with_synoy[gene_list_with_synoy["Synonyms_separated"].isin(gene_list_2_1)]
gene_list_df2_1


length of gene_list_2_1 5


,Symbol,Synonyms_separated
14017,FAM200C,ZBED8
17075,SCAND3,ZBED9
27751,GGT2P,GGT2
41677,LOC102724197,GGT2


In [40]:
gene_list_2_1

['Index', 'ZBED9', 'cancer_code', 'ZBED8', 'GGT2']

In [51]:
set(cho_et_al_tpm.columns.tolist())

{'RPL15',
 'SRGAP2',
 'APITD1-CORT',
 'DKK2',
 'CRH',
 'CCNY',
 'ZC3H13',
 'NAA16',
 'DNAAF3',
 'FAM181B',
 'PSPH',
 'ACAP2',
 'ZSCAN30',
 'SENP2',
 'B4GALT7',
 'HELB',
 'IMMP2L',
 'HTT',
 'FMOD',
 'CNGB1',
 'PHKG1',
 'NDUFB1',
 'GAPDHS',
 'SPICE1',
 'MYF5',
 'CD200R1L',
 'PF4V1',
 'ZSCAN1',
 'SRRT',
 'SPCS3',
 'TEX12',
 'C8orf74',
 'KBTBD6',
 'AKNAD1',
 'LARP4B',
 'SFXN5',
 'FMO5',
 'PSMB7',
 'SDCCAG3',
 'NVL',
 'NODAL',
 'PRB4',
 'OR3A1',
 'LAMTOR4',
 'OXSM',
 'ACTR1A',
 'NR5A2',
 'LAT2',
 'GMEB2',
 'LSM1',
 'PRMT2',
 'PDE7A',
 'ALS2CR11',
 'SCN8A',
 'RIPK4',
 'PTPRK',
 'SMCHD1',
 'RBMXL3',
 'SPTAN1',
 'SLC25A19',
 'LCA5L',
 'LRRC58',
 'UFM1',
 'CCDC69',
 'BTC',
 'KIAA1143',
 'PLAC8',
 'SLITRK6',
 'GEMIN2',
 'BCOR',
 'B3GAT1',
 'NPIPA3',
 'AQP4',
 'THNSL1',
 'CDKL1',
 'UBE2NL',
 'ZMYND11',
 'NCKAP1',
 'TRAF2',
 'C2orf66',
 'MTHFD2L',
 'FASTKD1',
 'YAF2',
 'MUT',
 'EPO',
 'NRN1L',
 'DDX60L',
 'PCDHB10',
 'ANO2',
 'ANO6',
 'OSBPL6',
 'CNOT1',
 'DSG1',
 'EBAG9',
 'UPK1B',
 'FAM104A',
 '

In [55]:
set(gene_list_df1_1["Synonyms_separated"].tolist())

{'-',
 '61E3.4',
 'AAP1',
 'AB13',
 'AB14',
 'AB23',
 'ACPL2',
 'AD-004',
 'AD026',
 'AD2',
 'AIPDS',
 'APAP-1',
 'ASXH2',
 'AZ1',
 'AZ2',
 'AZI1',
 'B7-H5',
 'B7H5',
 'BJHCC20A',
 'BRCC1',
 'BRP16',
 'BRP16L',
 'C10orf112',
 'C10orf118',
 'C10orf137',
 'C10orf54',
 'C11orf35',
 'C11orf82',
 'C12orf23',
 'C12orf39',
 'C12orf44',
 'C12orf52',
 'C12orf68',
 'C14orf164',
 'C15orf38',
 'C15orf60',
 'C16orf29',
 'C17orf72',
 'C17orf76',
 'C19orf55',
 'C19orf59',
 'C19orf69',
 'C19orf77',
 'C1orf163',
 'C1orf170',
 'C1orf172',
 'C1orf173',
 'C1orf227',
 'C1orf51',
 'C1orf63',
 'C1orf65',
 'C20orf105',
 'C20orf112',
 'C20orf113',
 'C20orf131',
 'C20orf132',
 'C20orf201',
 'C22orf36',
 'C22orf43',
 'C2orf85',
 'C3orf44',
 'C4orf21',
 'C6orf84',
 'C7orf10',
 'C7orf41',
 'C8orf30A',
 'C8orf30B',
 'C8orf47',
 'C9orf169',
 'C9orf96',
 'CCBP1',
 'CCCP-1',
 'CCCP1',
 'CCDC165',
 'CCDC41',
 'CCSP1',
 'CD234',
 'CDG1Z',
 'CDG2W',
 'CEBPZ-AS1',
 'CEMIP1',
 'CENP50',
 'CENPU50',
 'CFAP9',
 'CGI-110',
 '

In [ ]:
mapping_list = list(set(cho_et_al_tpm.columns.tolist())&set(gene_list_df1_1["Synonyms_separated"].tolist()))

print("length of mapping list", len(mapping_list))




length of mapping list 0


In [ ]:
genes_mapping = gene_list_df1_1[gene_list_df1_1["Synonyms_separated"].isin(mapping_list)]
genes_mapping_dict = {}
for index, row in genes_mapping.iterrows():
    genes_mapping_dict[row["Synonyms_separated"]] = row["Symbol"]
# add the other one which has synonyms in the list  
genes_mapping_dict

In [22]:
for index, row in gene_list_df2_1.iterrows():
    genes_mapping_dict[row["Symbol"]] = row["Synonyms_separated"]
genes_mapping_dict


{'LRMP': 'IRAG2',
 'WARS': 'WARS1',
 'YARS': 'YARS1',
 'GCKR': 'MAP4K5',
 'NPIPB5': 'NPIPB3',
 'MAST1': 'CLASP1',
 'FAM19A5': 'TAFA5',
 'NAA38': 'LSM8',
 'FAM105A': 'OTULINL',
 'IL25': 'MYDGF',
 'IL27': 'MYDGF',
 'AZI2': 'AZIN2',
 'FAM150B': 'ALKAL2',
 'TMEM173': 'STING1',
 'FAM159A': 'SHISAL2A',
 'LHFPL4': 'LHFPL3',
 'FAM150A': 'ALKAL1',
 'CAD': 'ACOD1',
 'FAM200C': 'ZBED8',
 'SCAND3': 'ZBED9',
 'GGT2P': 'GGT2',
 'LOC102724197': 'GGT2'}

In [24]:
cho_et_al_tpm = cho_et_al_tpm.rename(columns=genes_mapping_dict)
# add dummy value to the group for genes
new_mapping_list = list(set(compass_example_data_genelist) - set(cho_et_al_tpm.columns.tolist()))

new_mapping_list


['SAMD1',
 'CENPU',
 'C19orf84',
 'SF3B6',
 'AK6',
 'RFX7',
 'RYBP',
 'FTCDNL1',
 'GGT2',
 'ANKRD36C',
 'PLSCR3',
 'C3orf49',
 'NEURL1',
 'LKAAEAR1',
 'JADE2',
 'CEBPZOS',
 'C8orf88',
 'AZI2',
 'RAB29',
 'NAA38',
 'MS4A4E',
 'PIDD1',
 'ARPIN',
 'SMIM24',
 'CCDC184',
 'TMIGD3',
 'MTERF3',
 'GCNT7',
 'RSRP1',
 'ERVMER34-1',
 'ZNF670-ZNF695',
 'GCKR',
 'MTURN',
 'MUC2',
 'CEP131',
 'NOL4L',
 'NPIPB5',
 'LRRC75B',
 'LHFPL4',
 'CIPC',
 'ERICH3',
 'MAST1',
 'PRMT9',
 'RPEL1',
 'LRRC75A',
 'PRR29',
 'OTULIN',
 'HGH1',
 'SARAF',
 'ICE2',
 'NIFK',
 'ERICH5',
 'TMEM262',
 'MYO15B',
 'ZNF738',
 'NPIPB11',
 'RITA1',
 'MTERF1',
 'MALRD1',
 'COL6A5',
 'ANKRD18B',
 'CXCL8',
 'PROSER3',
 'CAD',
 'ACKR1',
 'SLC37A4',
 'ZPR1',
 'LMNTD1',
 'NPIPB4',
 'CT55',
 'ZBED9',
 'TMEM263',
 'HMCN2',
 'KDF1',
 'CEP162',
 'SPATA45',
 'NAPRT',
 'MTCL1',
 'Index',
 'DRICH1',
 'cancer_code',
 'CCDC185',
 'ZNF598',
 'NT5DC4',
 'ATG101',
 'C1orf167',
 'MCEMP1',
 'ZNF852',
 'CEMIP',
 'ZCCHC8',
 'RTP5',
 'ZGRF1',
 'MTERF2'

In [ ]:
cho_et_al[new_mapping_list] = 0.00
cho_et_al


In [ ]:
cho_et_al.shape


In [ ]:
cho_et_al_cleaned = cho_et_al.loc[:, cho_et_al.columns.isin(compass_example_data_genelist)]
cho_et_al_cleaned


In [ ]:
cho_et_al_cleaned["cancer_code"] = 16 # TCGA isn't having a broad type code but uses LUAD 
cho_et_al_cleaned = cho_et_al_cleaned[compass_example_data_genelist]
cho_et_al_cleaned = cho_et_al_cleaned.drop(columns="Index")


In [ ]:
cho_et_al_cleaned = cho_et_al_cleaned.drop(columns="cancer_code")
cho_et_al_cleaned


In [ ]:
# map the genes with gene id
# file from COMPASS-web
genes_mapping_compass = pd.read_csv("/Users/z5155527/Desktop/Benchmark-2025-Sep/phase_1_datasets/bulk_RNA-seq/compass/compass_gene_map.csv")
genes_mapping_compass


In [ ]:
mapping_id_dict = {}
mapping_id_dict_back = {}
for index, row in genes_mapping_compass.iterrows():
    mapping_id_dict[row["gene_name"]] = row["ensid_v36"]
    mapping_id_dict_back[row["ensid_v36"]] = row["gene_name"]
mapping_id_dict_back


In [ ]:
# rename
cho_et_al_cleaned = cho_et_al_cleaned.rename(columns=mapping_id_dict)
cho_et_al_cleaned


In [ ]:
# then transform back 
cho_et_al_tpm = pd.read_csv('/Users/z5155527/Desktop/Benchmark-2025-Sep/phase_1_datasets/bulk_RNA-seq/compass/unstranded_tpm.csv')
cho_et_al_tpm = cho_et_al_tpm.rename(columns={"Unnamed: 0": "Index"})
cho_et_al_tpm = cho_et_al_tpm.rename(columns=mapping_id_dict_back)
cho_et_al_tpm = cho_et_al_tpm[compass_example_data_genelist]
cho_et_al_tpm


In [ ]:
code_16_patients = cho_et_al_clinical.loc[cho_et_al_clinical["Histology"]=="Adenocarcinoma", "ID"]
cho_et_al_tpm.loc[cho_et_al_tpm["Index"].isin(code_16_patients), "cancer_code"] = 16
code_17_patients = cho_et_al_clinical.loc[cho_et_al_clinical["Histology"]=="Squamous_cell_carcinoma", "ID"]
cho_et_al_tpm.loc[cho_et_al_tpm["Index"].isin(code_17_patients), "cancer_code"] = 17
cho_et_al_tpm


In [ ]:
tpm = TPM(gtf_path).set_output(transform="pandas")
tmm = TMM(m_trim = 0.3, a_trim = 0.05).set_output(transform="pandas")


## counts, TPM, TMM
df_counts = cho_et_al_cleaned
df_tpm = tpm.fit_transform(df_counts)
df_tmm = tmm.fit_transform(df_counts)

df_counts.to_csv('/Users/z5155527/Desktop/Benchmark-2025-Sep/phase_1_datasets/bulk_RNA-seq/compass/untranded_counts.csv')
df_tpm.to_csv('/Users/z5155527/Desktop/Benchmark-2025-Sep/phase_1_datasets/bulk_RNA-seq/compass/unstranded_tpm.csv')


In [ ]:
# save the final output
cho_et_al_tpm.to_csv("/Users/z5155527/Desktop/Benchmark-2025-Sep/phase_1_datasets/bulk_RNA-seq/compass/cho_et_al_cleaned_TPM.tsv", sep='\t', index=False)


In [ ]:
cho_et_al_clinical["response_binary"] = 0
cho_et_al_clinical.loc[cho_et_al_clinical["Responsiveness"]=="Responder", "response_binary"] = 1
cho_et_al_clinical


In [ ]:
cho_et_al_clinical.to_csv("/Users/z5155527/Desktop/Benchmark-2025-Sep/phase_1_datasets/bulk_RNA-seq/compass/cho_et_al_clinical.tsv", sep='\t', index=False)
